## Some heading to talk about project

In [12]:
import numpy as np
import pptk

# Load the point cloud data
target_data = []
filename = 'ENVISAT.fbx.xyz'
with open(filename,'r') as f:
    for line in f.readlines():
        line_data = line.split()
        x, y, z = line_data
        target_data.append([float(x), float(y), float(z)])
target_data = np.asarray(target_data)

In [39]:
# Visualize the target model
v = pptk.viewer(target_data)

### Generating Point Cloud Data


In [33]:
# Transformation matrices
def transform(phi, theta, gamma):
    ''' Creates the transform matrix R
    ARGS:
        phi (rad):      rotation about x-axis
        theta (rad):    rotation about y-axis
        gamma(rad):     rotation about z-axis
    '''
    R_x = np.array([
        [1,  0,            0          ],
        [0,  np.cos(phi),  np.sin(phi)],
        [0, -np.sin(phi),  np.cos(phi)]
    ])
    R_y = np.array([
        [np.cos(theta), 0, -np.sin(theta)],
        [0,             1,  0            ],
        [np.sin(theta), 0,  np.cos(theta)]
    ])
    R_z = np.array([
        [ np.cos(gamma), np.sin(gamma), 0],
        [-np.sin(gamma), np.cos(gamma), 0],
        [0,              0,             1]
    ])

    R = np.cross(np.cross(R_y, R_x), R_z)

    return R

def simulated_positin(xyz, T_0, R):
    ''' Given target model computes the simulated sensor data
    ARGS:
        xyz (Nx3):  point cloud data of the target model
        T_0 (3x1):  relative position
    '''
    T = np.zeros([3,1])
    new_point_cloud = np.zeros(xyz.shape)
    row = 0
    for entry in xyz:
        # Each entry contains x,y,z
        entry_T = entry.reshape(3,1) # transpose data into column vector
        T = T_0 + entry_T
        sim = np.dot(R, entry_T) + T
        new_point_cloud[row,:] = sim.reshape(1,3)
        row += 1

    return new_point_cloud

In [41]:
# Reproducing Experiment 1 - 
#   (x,y,z) = (10,0,0) meters
#   (phi, theta, gamma) = (-180,0,0) degrees
T_0 = np.array([ [33], [0], [0] ])

R = transform(np.deg2rad(-180), 0, 0)
new_point_cloud = simulated_positin(target_data, T_0, R)
v = pptk.viewer(new_point_cloud)